# Preparam el notebook

In [1]:
import os
import time
import itertools

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import image
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV, train_test_split

from skimage.feature import hog
from skimage import io
from skimage.transform import rotate, resize
from skimage.util import random_noise
from scipy import ndimage

# Llegim totes les imatges del dataset

In [2]:
categories_to_exclude = ['livingroom (case conflict)']

def get_image_files_by_folder(root_folder, image_extensions=('jpg', 'jpeg', 'png', 'gif', 'bmp')):
    image_files_by_folder = {}

    for folder_path, _, files in os.walk(root_folder):
        # Aseguram que les categories tinguin el mateix nom
        folder_name = os.path.basename(folder_path).lower()
        image_files = []

        if folder_name in categories_to_exclude:
            continue
        
        for file in files:
            file_extension = file.lower().split('.')[-1]
            if file_extension in image_extensions:
                image_files.append(os.path.join(folder_path, file))

        if image_files:
            image_files_by_folder[folder_name] = image_files

    return image_files_by_folder

images_to_load_train = get_image_files_by_folder('data/train')
images_to_load_test = get_image_files_by_folder('data/test')

# Cream els dataframes

In [3]:
def read_and_flatten_images(image_list):
    image_data = []
    
    for image_path in image_list:
        img = io.imread(image_path, as_gray=True)
        img = resize(img, (200, 200))
        image_data.append(img.flatten())
    
    return image_data

def create_dataframe_from_categories(images_to_load):
    df = pd.DataFrame()
    for label, image_list in images_to_load.items():
        image_data = read_and_flatten_images(images_to_load[label])
        next_part = pd.DataFrame(image_data)
        next_part['label'] = label
        df = pd.concat([df, next_part], ignore_index=True)
    return df

train_df = create_dataframe_from_categories(images_to_load_train)
test_df = create_dataframe_from_categories(images_to_load_test)

In [4]:
train_df_no_label = train_df.drop('label', axis=1)
train_labels = train_df['label']

def fit_predict_svm(X_train, X_test, y_train, y_test):
    svm = SVC(kernel='linear', random_state=1)
    svm.fit(X_train, y_train)
    y_predicted = svm.predict(X_test)
    print(f"La precisició es de: {accuracy_score(y_test, y_predicted)}")
    
    return y_predicted

def try_hog(*args, **kwargs):
    def apply_hog(row):
        image = row.values.reshape(200, 200)
        return hog(image, **kwargs)

    start_time = time.time()
    hog_df = train_df_no_label.apply(apply_hog, axis=1, result_type='expand')
    print(f"Dimensions del DF: {hog_df.shape}")

    X_train, X_validation, y_train, y_validation = train_test_split(hog_df, train_labels, test_size=0.20, random_state=1)
    fit_predict_svm(X_train, X_validation, y_train, y_validation)
    print("HOG + Fit time:", time.time() - start_time, "seconds")

# Entrenam el model i donam una predicció amb diferents paràmetres del HOG

In [5]:
try_hog()

Dimensions del DF: (1500, 42849)
La precisició es de: 0.6533333333333333
HOG + Fit time: 48.27572226524353 seconds


In [6]:
# Normalitzat?
try_hog(transform_sqrt=True)

Dimensions del DF: (1500, 42849)
La precisició es de: 0.67
HOG + Fit time: 48.17511796951294 seconds


In [7]:
# Augmentar orientacions?
try_hog(orientations=12, transform_sqrt=True)

Dimensions del DF: (1500, 57132)
La precisició es de: 0.6666666666666666
HOG + Fit time: 99.12308979034424 seconds


In [8]:
# Reduir orientancions?
try_hog(orientations=8, transform_sqrt=True)

Dimensions del DF: (1500, 38088)
La precisició es de: 0.6366666666666667
HOG + Fit time: 80.5388879776001 seconds


In [9]:
# Reduir orientancions?
try_hog(orientations=6, transform_sqrt=True)

Dimensions del DF: (1500, 28566)
La precisició es de: 0.63
HOG + Fit time: 67.32906126976013 seconds


In [10]:
# Menys resolucio?
try_hog(orientations=9, pixels_per_cell=(10, 10), cells_per_block=(5, 5), transform_sqrt=True)

Dimensions del DF: (1500, 57600)
La precisició es de: 0.65
HOG + Fit time: 92.13097381591797 seconds


In [11]:
try_hog(orientations=9, pixels_per_cell=(10, 10), cells_per_block=(10, 10), transform_sqrt=True)

Dimensions del DF: (1500, 108900)
La precisició es de: 0.6
HOG + Fit time: 146.9048833847046 seconds


In [12]:
try_hog(orientations=9, pixels_per_cell=(20, 20), cells_per_block=(10, 10))

Dimensions del DF: (1500, 900)
La precisició es de: 0.5666666666666667
HOG + Fit time: 14.77531909942627 seconds


In [13]:
try_hog(orientations=9, pixels_per_cell=(20, 20), cells_per_block=(10, 10), transform_sqrt=True)

Dimensions del DF: (1500, 900)
La precisició es de: 0.64
HOG + Fit time: 12.123485088348389 seconds


# Es pot reduir més la resolució del HOG?

In [14]:
try:
    try_hog(orientations=9, pixels_per_cell=(20, 20), cells_per_block=(11, 11), transform_sqrt=True)
except ValueError:
    print("Error, no es pot augmentar més el tamany de cells_per_block")

Dimensions del DF: (1500, 0)
Error, no es pot augmentar més el tamany de cells_per_block


In [15]:
try:
    try_hog(orientations=9, pixels_per_cell=(21, 21), cells_per_block=(10, 10), transform_sqrt=True)
except ValueError:
    print("Error, no es pot augmentar més el tamany de pixels_per_cell")

Dimensions del DF: (1500, 0)
Error, no es pot augmentar més el tamany de pixels_per_cell


# Conclusions

El HOG més òptim en termes de precisió és el que fa servir els paràmetres per defecte però aplicant normalització. Tot això, podem veure que és unes 6,5 vegades més lent que un HOG de més baixa resolució com és `pixels_per_cell=(20, 20), cells_per_block=(10, 10)`.

És per això que s'ha triat sacrificar un 3% de precisió a canvi d'un millor temps d'execució que permet fer cerques més exhaustives dels hiperparàmetres de l'SVM. El model final hauria de fer servir la parametrització per defecte per a millor rendiment.